In [1]:
import base64
import os
import ssl
import re
import random
from urllib import request
import cv2
import imageio
import numpy as np
import scipy.io.wavfile
import ffmpeg
import shutil

from IPython.display import HTML

In [2]:
UCF_ROOT_URL = 'https://www.crcv.ucf.edu/THUMOS14/UCF101/UCF101/'
context = ssl._create_unverified_context()

def ucf_label2name_dict():
  idx = request.urlopen(UCF_ROOT_URL, context=context).read().decode('utf-8')
  videos = sorted(list(set(re.findall('(v_[\w_]+\.avi)', idx))))
  output = {}
  for video in videos:
    label = re.findall('v_(.*)_g', video)[0]
    output.setdefault(label, []).append(video)
  return output

def fetch_ucf_video(video, destination):
  urlpath = request.urljoin(UCF_ROOT_URL, video)
  print(f'Fetching {urlpath} -> {destination}')
  data = request.urlopen(urlpath, context=context).read()
  open(destination, "wb").write(data)
  return destination

def prepare_ucf_dataset(destination='ucf101', num_per_class=None, random_sample=False):
  cnt = 0
  for label, videos in ucf_label2name_dict().items():
    if num_per_class:
      if random_sample:
        videos = random.sample(videos, num_per_class)
      else:
        videos = videos[:num_per_class]
    for video in videos:
      data_name = re.findall('(.*).avi', video)[0]
      data_dir = os.path.join(destination, label, data_name)
      os.makedirs(data_dir, exist_ok=True)
      video_dest = os.path.join(data_dir, video)
      audio_dest = os.path.join(data_dir, video.replace('.avi', '.wav'))
      if not os.path.exists(video_dest):
        fetch_ucf_video(video, video_dest)
        try:
          ffmpeg.run(ffmpeg.output(ffmpeg.input(video_dest), audio_dest))
        except:
          shutil.rmtree(os.path.join(destination, label))
          break
        cnt += 1
  return cnt

1.5h

In [8]:
print('video count:', prepare_ucf_dataset(num_per_class=200))

ffmpeg version 4.2.7-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --e

Fetching https://www.crcv.ucf.edu/THUMOS14/UCF101/UCF101/v_TableTennisShot_g24_c01.avi -> ucf101/TableTennisShot/v_TableTennisShot_g24_c01/v_TableTennisShot_g24_c01.avi


ffmpeg version 4.2.7-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --e

Fetching https://www.crcv.ucf.edu/THUMOS14/UCF101/UCF101/v_TableTennisShot_g24_c02.avi -> ucf101/TableTennisShot/v_TableTennisShot_g24_c02/v_TableTennisShot_g24_c02.avi


ffmpeg version 4.2.7-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --e

...

ffmpeg version 4.2.7-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --e

Fetching https://www.crcv.ucf.edu/THUMOS14/UCF101/UCF101/v_WritingOnBoard_g25_c07.avi -> ucf101/WritingOnBoard/v_WritingOnBoard_g25_c07/v_WritingOnBoard_g25_c07.avi


ffmpeg version 4.2.7-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --e

Fetching https://www.crcv.ucf.edu/THUMOS14/UCF101/UCF101/v_YoYo_g01_c01.avi -> ucf101/YoYo/v_YoYo_g01_c01/v_YoYo_g01_c01.avi
video count: 4173


ffmpeg version 4.2.7-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --e

In [3]:
for label in sorted(os.listdir('ucf101')):
    print(label, len(os.listdir(os.path.join('ucf101', label))))

ApplyEyeMakeup 50
ApplyLipstick 50
Archery 50
BabyCrawling 50
BalanceBeam 50
BandMarching 50
BasketballDunk 50
BlowDryHair 50
BlowingCandles 50
BodyWeightSquats 50
Bowling 50
BoxingPunchingBag 50
BoxingSpeedBag 50
BrushingTeeth 50
CliffDiving 50
CricketBowling 50
CricketShot 50
CuttingInKitchen 50
FieldHockeyPenalty 50
FloorGymnastics 50
FrisbeeCatch 50
FrontCrawl 50
Haircut 50
HammerThrow 50
Hammering 50
HandstandPushups 50
HandstandWalking 50
HeadMassage 50
IceDancing 50
Knitting 50
LongJump 50
MoppingFloor 50
ParallelBars 50
PlayingCello 50
PlayingDaf 50
PlayingDhol 50
PlayingFlute 50
PlayingSitar 50
Rafting 50
ShavingBeard 50
Shotput 50
SkyDiving 50
SoccerPenalty 50
StillRings 50
SumoWrestling 50
Surfing 50
TableTennisShot 50
Typing 50
UnevenBars 50
WallPushups 50
WritingOnBoard 50


In [4]:
import torch
import torchaudio
import matplotlib.pyplot as plt

def crop_center_square(frame):
  h, w = frame.shape[0:2]
  min_dim = min(h, w)
  sx = (w//2) - (min_dim//2)
  sy = (h//2) - (min_dim//2)
  return frame[sy : sy+min_dim, sx : sx+min_dim]

def load_video(path, max_time=None, resize=(224, 224)):
  cap = cv2.VideoCapture(path)
  frames=[]
  try:
    while True:
      ret, frame = cap.read()
      if not ret: break
      frame = crop_center_square(frame)
      # frame = cv2.resize(frame, resize)
      frame = frame[:, :, [2, 1, 0]]
      frames.append(frame)
      if max_time:
        if cap.get(cv2.CAP_PROP_POS_MSEC) * 1e-3 >= max_time:
          break
  finally:
    cap.release()
  video_tensor = torch.tensor(frames) / 255.0
  return torch.permute(video_tensor, (3, 0, 1, 2)) #THWC -> CTHW

def load_audio(path, max_time=None):
  waveform, sample_rate = torchaudio.load(path)
  if max_time:
    max_frame = int(max_time * sample_rate)
    waveform = waveform[:, :max_frame]
  return waveform, sample_rate


/home/rkn/.local/share/virtualenvs/lab-work-gFKfzxI5/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
load_video('ucf101/ApplyEyeMakeup/v_ApplyEyeMakeup_g01_c01/v_ApplyEyeMakeup_g01_c01.avi').shape

/tmp/ipykernel_103716/684986552.py:28: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:230.)
  video_tensor = torch.tensor(frames) / 255.0


torch.Size([3, 164, 240, 320])

In [6]:
import torchvision
from torchvision.datasets import DatasetFolder
from torch.utils.data import Dataset

def loader(path, max_time):
  datum = os.listdir(path)
  for data in datum:
    if data.endswith('.avi'):
      video = load_video(os.path.join(path, data), max_time=max_time)
    elif data.endswith('.wav'):
      audio, _ = load_audio(os.path.join(path, data), max_time=max_time)
  return video, audio


class UCFDataset(DatasetFolder):
  def __init__(self, root, loader, max_time=None):
    self.root = root
    self.loader = loader
    self.max_time = max_time
    self.classes, self.class_to_idx = super().find_classes(root)
    self.samples = self.make_dataset(self.root, self.class_to_idx)

  def make_dataset(self, directory, class_to_idx):
    instances = []
    for target_class in sorted(class_to_idx.keys()):
      class_idx = class_to_idx[target_class]
      target_dir = os.path.join(directory, target_class)
      if not os.path.isdir(target_dir):
        continue
      for data_name in sorted(os.listdir(target_dir)):
        path = os.path.join(target_dir, data_name)
        item = path, class_idx
        instances.append(item)
    return instances

  def __getitem__(self, idx):
    path, target = self.samples[idx]
    sample = self.loader(path, self.max_time)
    return sample, target


class SetTransform(Dataset):
  def __init__(self, dataset, transform):
    self.dataset = dataset
    self.transform = transform

  def __getitem__(self, idx):
    (video, audio), label = self.dataset[idx]
    video = self.transform(video)
    return (video, audio), label

  def __len__(self):
    return len(self.dataset)


In [7]:
from torchvision.transforms import (Compose,
                                    Resize)

transforms = Compose([
    Resize((224, 224)),
])

all_dataset = UCFDataset('ucf101', loader=loader, max_time=5)
len(all_dataset)

2550

In [ ]:
val_size = 1000
test_size = 1000
train_size = len(all_dataset) - val_size - test_size

test_dataset, trainval_dataset = torch.utils.data.random_split(all_dataset, [test_size, train_size + val_size])
train_dataset, val_dataset = torch.utils.data.random_split(trainval_dataset, [train_size, val_size])

train_dataset = SetTransform(train_dataset, transforms)
val_dataset = SetTransform(val_dataset, transforms)
test_dataset = SetTransform(test_dataset, transforms)

print('train:', len(train_dataset))
print('validation:', len(val_dataset))
print('test:', len(test_dataset))

In [76]:
(video, audio), label = all_dataset[0]
print(video.shape)
print(audio.shape)
print(label)

torch.Size([125, 240, 320, 3])
torch.Size([2, 220500])
0


In [ ]:
def collate_fn(examples):
    (videos, audios), labels = zip(*examples)
    videos = torch.stack(videos)
    audios = torch.stack(audios)
    labels = torch.tensor(labels)
    return {'videos': videos, 'audios': audios, 'labels': labels}

In [ ]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_dataset, collate_fn=collate_fn, batch_size=4)

batch = next(iter(train_dataloader))
for k,v in batch.items():
    if isinstance(v, torch.Tensor):
        print(k, v.shape)